In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#Imports:
from comet_ml import Experiment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import random
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB,CategoricalNB
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import re
from nltk.corpus import stopwords
import string
from sklearn import preprocessing
from sklearn.manifold import TSNE
import seaborn as sns
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import svm
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from time import time
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
import optuna
sns.set(context='notebook', style='darkgrid', palette='colorblind', font='sans-serif', font_scale=1, rc=None)
matplotlib.rcParams['figure.figsize'] =[8,8]
matplotlib.rcParams.update({'font.size': 15})
matplotlib.rcParams['font.family'] = 'sans-serif'

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bartalisd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bartalisd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
#Load data:
with open("../data_root.txt") as f:
    data_root_dir = f.read()
data_root_dir
train = pd.read_csv('%s/tweet_disaster/train.csv' % data_root_dir)
test = pd.read_csv('%s/tweet_disaster/test.csv' % data_root_dir)
#sub = pd.read_csv('../input/nlp-getting-started/sample_submission.csv')

In [4]:
#Analyse & clean data:
from data_analysis import analysis, data_cleaning
frame, train, vocabulary = analysis(train, test)
X, Y = data_cleaning(train, test)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bartalisd/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bartalisd/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/home/bartalisd/git/apricot_exp/experiments/tweet_disaster/data_analysis.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text_length'] = train.text.apply(lambda x: len(x.split()))


In [5]:
#Train-test cut:
from apricot_exp.evaluation import traintest
X_train, X_test, Y_train, Y_test = traintest(X,Y,0.3)

In [6]:
#TFIDF & transpose data:
max_num_features = 10000
from data_analysis import tfidf
features_t, features_test_t = tfidf(X_train, X_test, max_num_features)
print(features_t.shape)
print(features_test_t.shape)

(10000, 5264)
(10000, 2257)


In [7]:
from apricot_exp.comet_utils import *
api_key = load_api_key('../../comet_key.txt')

In [8]:
#Bare model:
model = LogisticRegression(max_iter=1000)
n = 9000
experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
experiment.log_parameters({
    'size': None,
    'function': None,
})
tfidf = TfidfVectorizer(sublinear_tf=True,max_features=n, min_df=1, norm='l2',  ngram_range=(1,2))
features = tfidf.fit_transform(X_train).toarray()
features_test = tfidf.transform(X_test).toarray()
print(features.shape, features_test.shape)
from apricot_exp.evaluation import train_eval
train_eval(model, features,  Y_train, features_test, Y_test, experiment)
experiment.end()

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/ecbefb7ccf5a423f837b3fbb0ac5c67d



(5264, 9000) (2257, 9000)


COMET INFO: Uploading stats to Comet before program termination (may take several seconds)
COMET INFO: Still uploading


In [9]:
from apricot_exp.func_tp import featureb, facilityloc, maxcov
from comet import suggest_config, extract_grid

In [10]:
from optuna.samplers import TPESampler, RandomSampler, GridSampler
from parameters import param, algo
from apricot_exp.evaluation import train_eval

In [11]:
#Feature-based(naive, lazy, two-stage running time)
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    for j in ["naive", "lazy", "two-stage"]:
        experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
        n = i    
        start_time = time.time()
        Xtr_t, Xte_t = featureb(features_t, features_test_t, i, "sqrt", j)
        experiment.log_metric("running_time", time.time()-start_time)
        Xtr = Xtr_t.transpose()
        Xte = Xte_t.transpose()
        acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        print(j)
        print(time.time()-start_time)
        experiment.log_parameters({
            'optimizer': j,
            'size': i,
            'function': 'featurebased'
        })
        experiment.end() 

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/7d6f78b719ff48de9a0c8abd517b662f

COMET INFO: Still uploading


naive
8.883957386016846


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/c58ae00e7aa74150b48461585db4fb84

COMET INFO: Still uploading


lazy
5.935347080230713


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/4aee88119e8341ee89127ac68224290a

COMET INFO: Still uploading


...

naive
2465.3778603076935


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/b98ac3748889422587c16a8607e4da9a

COMET INFO: Still uploading


lazy
2196.1041157245636


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/7b6db85758e8479aa24cf4cc5917d1d4

COMET INFO: Still uploading


two-stage
2541.78133559227


In [ ]:
#Feature-based(approximate-lazy, stochastic, sample)
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    for j in ["approximate-lazy", "stochastic", "sample"]:
        experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
        n = i    
        start_time = time.time()
        Xtr_t, Xte_t = featureb(features_t, features_test_t, i, "sqrt", j)
        experiment.log_metric("running_time", time.time()-start_time)
        Xtr = Xtr_t.transpose()
        Xte = Xte_t.transpose()
        acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        print(j)
        print(time.time()-start_time)
        experiment.log_parameters({
            'optimizer': j,
            'size': i,
            'function': 'featurebased'
        })
        experiment.end() 

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/0c5299ff6a8c43f39153d4a157453532

COMET INFO: Still uploading


approximate-lazy
6.62127161026001


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/f858edb6e4e7458ea1a8c13c7feb2529

COMET INFO: Still uploading


stochastic
8.08964228630066


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/8209bd5b3d7542e786c6654d7e742e79

COMET INFO: Still uploading


...

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/53d5ff09e5f04caea767c1ad8b7ac2a8

COMET INFO: Still uploading


sample
2026.6739783287048


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/5350df9c676f4844ae817b1211786ba7

COMET INFO: Still uploading


approximate-lazy
2503.0214142799377


COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/797db273b9b64810bdf54ef924dc05b4



In [ ]:
parameters = param('featurebased')
search_alg = "GRID"
algo = algo(search_alg, parameters)
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    n = i    
    def objective(trial):
        config = suggest_config(parameters, trial)
        roc=0.0
        try:
            start_time = time.time()
            Xtr_t, Xte_t = featureb(features_t, features_test_t, i, config["function"], config["optimizer"])
            experiment.log_metric("running_time", time.time()-start_time)
            Xtr = Xtr_t.transpose()
            Xte = Xte_t.transpose()
            acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        except Exception as err:
            print(err)
        finally:   
            return roc
    study = optuna.create_study(direction="maximize", sampler=algo)
    study.optimize(objective, n_trials=20, n_jobs=1)
    best_param = study.best_params
    print(best_param)
    experiment.log_parameters({
        'best param': best_param,
        'size': i,
        'function': 'featurebased'
    })
    experiment.end()

In [11]:
parameters = param('facilitylocation')
search_alg = "RND"
algo = algo(search_alg, parameters)
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    n = i    
    def objective(trial):
        config = suggest_config(parameters, trial)
        roc=0.0
        try:
            start_time = time.time()
            Xtr_t, Xte_t = facilityloc(features_t, features_test_t, i, config["function"], "lazy")
            experiment.log_metric("running_time", time.time()-start_time)
            Xtr = Xtr_t.transpose()
            Xte = Xte_t.transpose()
            acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        except Exception as err:
            print(err)
        finally:   
            return roc
    study = optuna.create_study(direction="maximize", sampler=algo)
    study.optimize(objective, n_trials=20, n_jobs=1)
    best_param = study.best_params
    print(best_param)
    experiment.log_parameters({
        'best param': best_param,
        'size': i,
        'function': 'facilitylocation'
    })
    experiment.end()

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/5f2b18ff6b38492d9bdd391200b7181d

[I 2021-11-15 21:01:44,753] A new study created in memory with name: no-name-14257891-c2e9-4d6c-9b91-10d6c563efe5
[I 2021-11-15 21:01:44,763] Trial 0 finished with value: 0.0 and parameters: {'function': 'l1'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:44,769] Trial 1 finished with value: 0.0 and parameters: {'function': 'l1'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:44,776] Trial 2 finished with value: 0.0 and parameters: {'function': 'cosine'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:44,789] Trial 3 finished with value: 0.0 and parameters: {'function': 'correlation'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:44,795] Trial 4 finished with value: 0.0 and parameters: {'function': 'cityblock'}. Best is 

initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subs

COMET INFO: Still uploading
COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/ff9a57d8b0564a22a9f30be244e9bf24

[I 2021-11-15 21:01:47,912] A new study created in memory with name: no-name-09debaf8-dd76-4027-902a-0dcbfee91f78
[I 2021-11-15 21:01:47,918] Trial 0 finished with value: 0.0 and parameters: {'function': 'correlation'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:47,923] Trial 1 finished with value: 0.0 and parameters: {'function': 'l1'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:47,929] Trial 2 finished with value: 0.0 and parameters: {'function': 'cityblock'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:47,935] Trial 3 finished with value: 0.0 and parameters: {'function': 'l2'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:47,940] Trial 4 finished with value: 0.0 and parameters: {'fu

initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subs

COMET INFO: Still uploading
COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/84af3ee3c4af4576b5793046af2eb9d0

[I 2021-11-15 21:01:51,368] A new study created in memory with name: no-name-a90e0dd9-ee8f-46f1-bc30-1c91efa66ba2
[I 2021-11-15 21:01:51,375] Trial 0 finished with value: 0.0 and parameters: {'function': 'l2'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:51,382] Trial 1 finished with value: 0.0 and parameters: {'function': 'manhattan'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:51,385] Trial 2 finished with value: 0.0 and parameters: {'function': 'cosine'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:51,391] Trial 3 finished with value: 0.0 and parameters: {'function': 'l1'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:01:51,396] Trial 4 finished with value: 0.0 and parameters: {'functio

...

COMET INFO: Still uploading
COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/8727ba49c5074af2b65cca7430518963

[I 2021-11-15 21:02:16,196] A new study created in memory with name: no-name-2034b1b1-a083-4e3c-9f47-0872c34111f5
[I 2021-11-15 21:02:16,204] Trial 0 finished with value: 0.0 and parameters: {'function': 'manhattan'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:16,211] Trial 1 finished with value: 0.0 and parameters: {'function': 'l2'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:16,217] Trial 2 finished with value: 0.0 and parameters: {'function': 'correlation'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:16,224] Trial 3 finished with value: 0.0 and parameters: {'function': 'euclidean'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:16,228] Trial 4 finished with value: 0.0 and parameter

initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subs

COMET INFO: Still uploading
COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/ea49387bfc6b4be9935cfcb6c66df657

[I 2021-11-15 21:02:22,761] A new study created in memory with name: no-name-426f35d3-0b13-416b-8b30-11aa9b25e471
[I 2021-11-15 21:02:22,768] Trial 0 finished with value: 0.0 and parameters: {'function': 'correlation'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:22,773] Trial 1 finished with value: 0.0 and parameters: {'function': 'euclidean'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:22,777] Trial 2 finished with value: 0.0 and parameters: {'function': 'l1'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:22,783] Trial 3 finished with value: 0.0 and parameters: {'function': 'l2'}. Best is trial 0 with value: 0.0.
[I 2021-11-15 21:02:22,787] Trial 4 finished with value: 0.0 and parameters: {'fu

initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subset must be a list, numpy array, or None
initial_subs

COMET INFO: Still uploading


In [ ]:
parameters = param('maxcoverage')
search_alg = "GRID"
algo = algo(search_alg, parameters)
for i in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]:
    experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
    experiment.log_parameters({
        'size': i,
        'function': 'maxcoverage'
    })
    n = i    
    def objective(trial):
        config = suggest_config(parameters, trial)
        roc=0.0
        try:
            start_time = time.time()
            Xtr_t, Xte_t = maxcov(features_t, features_test_t, i, "two_stage")
            experiment.log_metric("running_time", time.time()-start_time)
            Xtr = Xtr_t.transpose()
            Xte = Xte_t.transpose()
            acc, pre, rec, roc = train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        except Exception as err:
            print(err)
        finally:   
            return roc
    study = optuna.create_study(direction="maximize", sampler=algo)
    study.optimize(objective, n_trials=20, n_jobs=1)
    best_param = study.best_params
    print(best_param)
    experiment.log_parameters({
        'best param': best_param,
        'size': i,
        'function': 'maxcoverage'
    })
    experiment.end()

COMET WARNING: Config key 'comet.auto_log.weights' is deprecated, please use 'comet.auto_log.histogram_weights' instead
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/apricot/tweet-disaster/615ccb1b91d0406db168c51db259bac2

[I 2021-11-15 21:12:43,740] A new study created in memory with name: no-name-6ff6a5b4-6c09-48c9-b978-fff6b444b865
[I 2021-11-15 21:12:50,326] Trial 0 finished with value: 0.0 and parameters: {'optimizer': 'stochastic'}. Best is trial 0 with value: 0.0.


No matching definition for argument type(s) array(float64, 1d, C), array(int32, 1d, C), array(int32, 1d, C), array(float64, 1d, C), array(float64, 1d, C), unicode_type, array(int64, 1d, C)


[I 2021-11-15 21:12:56,845] Trial 1 finished with value: 0.0 and parameters: {'optimizer': 'bidirectional'}. Best is trial 0 with value: 0.0.


No matching definition for argument type(s) array(float64, 1d, C), array(int32, 1d, C), array(int32, 1d, C), array(float64, 1d, C), array(float64, 1d, C), unicode_type, array(int64, 1d, C)


[I 2021-11-15 21:13:03,040] Trial 2 finished with value: 0.0 and parameters: {'optimizer': 'random'}. Best is trial 0 with value: 0.0.


...

No matching definition for argument type(s) array(float64, 1d, C), array(int32, 1d, C), array(int32, 1d, C), array(float64, 1d, C), array(float64, 1d, C), unicode_type, array(int64, 1d, C)


[I 2021-11-15 21:21:26,213] Trial 2 finished with value: 0.0 and parameters: {'optimizer': 'approximate-lazy'}. Best is trial 0 with value: 0.0.


No matching definition for argument type(s) array(float64, 1d, C), array(int32, 1d, C), array(int32, 1d, C), array(float64, 1d, C), array(float64, 1d, C), unicode_type, array(int64, 1d, C)


[I 2021-11-15 21:21:32,434] Trial 3 finished with value: 0.0 and parameters: {'optimizer': 'stochastic'}. Best is trial 0 with value: 0.0.


No matching definition for argument type(s) array(float64, 1d, C), array(int32, 1d, C), array(int32, 1d, C), array(float64, 1d, C), array(float64, 1d, C), unicode_type, array(int64, 1d, C)


In [ ]:
def randomtrain(X_tr, X_te, n):
    X_train_arr = X_tr.to_numpy()
    X_test_arr = X_te.to_numpy() 
    print(X_train_arr.shape, X_test_arr.shape)
    idxs = np.arange(X_tr.shape[0])
    np.random.shuffle(idxs)
    idx = idxs[:n]
    Xtr, Xte = X_train_arr[idx,:], X_test_arr[idx,:]
    return Xtr, Xte

In [ ]:
for j in [100, 500, 1000, 2500, 5000, 7500, 8000, 9000]: 
    for i in range(1,20):
        n = j
        experiment=init_experiment(api_key, 'tweet_disaster', 'apricot')
        experiment.log_parameters({
            'size': i,
            'function': 'random'
        })
        Xtr_t, Xte_t = randomtrain(features_t, features_test_t, n)
        Xtr = Xtr_t.transpose()
        Xte = Xte_t.transpose()
        from apricot_exp.evaluation import train_eval
        train_eval(model, Xtr, Y_train, Xte, Y_test, experiment)
        i = i+1
        experiment.end()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(n_estimators=20, max_depth=3)

In [ ]:
from apricot_exp.evaluation import train_eval
train_eval(model, features,  Y_train, features_test, Y_test, experiment)

## PCA

In [ ]:
#pca = PCA(n_components=2)

In [ ]:
#from sklearn.preprocessing import StandardScaler
#f = list(range(0, 10000))
#x = features[:, f]
#x = StandardScaler().fit_transform(x)
#x = pd.DataFrame(x)

In [ ]:
#from sklearn.decomposition import PCA
#pca = PCA(n_components=100)
#x_pca = pca.fit_transform(x)
#x_pca = pd.DataFrame(x_pca)
#x_pca.head()

## Visualization

In [ ]:
experiment.display(tab="tweet disaster")

In [ ]:
import neptune
project = neptune.init(project_qualified_name='bartalisd/nlp-disaster-tweets',
             api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vdWkubmVwdHVuZS5haSIsImFwaV91cmwiOiJodHRwczovL3VpLm5lcHR1bmUuYWkiLCJhcGlfa2V5IjoiNmY5YjdlNTgtNjMxNC00ODQyLThiNGItNmM5NTBmZTM1MjA4In0=', 
            )

# Download experiments dashboard as pandas DataFrame
data = project.get_leaderboard()

data=data[data['tags'].map(len) > 1]
data = data.drop(['name','created', 'finished','owner','notes', 'running_time','size'], axis=1)
for i in ['channel_roc', 'channel_acc', 'channel_rec', 'channel_pre']:
    data[i]=data[i].astype("float64")

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
def barplot_res(dataframe, percentage, metric, y_name):
    selector = dataframe.apply(lambda x: percentage in x["tags"], axis=1)
    data_percentage=dataframe.loc[selector, :]
    data_percentage["tags"]=data_percentage["tags"].apply(lambda x: str(x))
    res_percentage=data_percentage.groupby(by='tags')[metric].mean().reset_index()
    res_percentage['model']=res_percentage['tags'].apply(lambda x: eval(x)[1])
    ax=sns.barplot(data=res_percentage, x='model', y=metric)
    plt.ylim((0.5, 0.9))
    ax.set(xlabel='Model', ylabel=y_name, title =percentage)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=60)
    return ax, res_percentage
def lineplot_res(dataframe, y_name, metric, res5k, mod):
    data=dataframe.copy()
    data['percentage']=data['tags'].apply(lambda x: x[1])
    data=data[~data['percentage'].apply(lambda x: 'd' in str(x))]
    data['model']=data['tags'].apply(lambda x: x[0])
    functions=['facilityloc_d', 'featurebased_d', 'maxcov_d', 'random_d', 'Tfidf']
    data=data[data['model'].isin(functions)]
    result=data.groupby(by=['model','percentage'])[metric].mean().reset_index()
    ax=sns.lineplot(data=result, x='percentage', y=metric, hue='model', palette=['yellow', 'b','r','forestgreen','blueviolet'  ])
    ax.set_title(mod,fontdict= { 'fontsize': 20, 'fontweight':'bold'})
    plt.xlabel('Number of features', fontsize= 15)
    plt.ylabel(y_name, fontsize= 15)
    plt.axhline(y=res5k, color='brown', linestyle='--')
    plt.setp(ax.get_legend().get_texts(), fontsize='15')
    plt.setp(ax.get_legend().get_title(), fontsize='20')
    return ax

In [ ]:
lineplot_res(data, 'Accuracy', 'channel_acc', 0.7984049623393886, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'Precision', 'channel_pre', 0.8259162303664922, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'Recall', 'channel_rec', 0.6621196222455404, 'NLP - Disasters tweets: Dim reduction' )

In [ ]:
lineplot_res(data, 'ROC AUC', 'channel_roc', 0.8622086211447222, 'NLP - Disasters tweets: Dim reduction' )